In [76]:
import numpy as np

# Read in and prepare text data

In [77]:
with open("goblet_book.txt", "r") as f:
    data = f.read()

book_chars = list(set(data))
K = len(book_chars)

charToInd = {c:i for i,c in enumerate(book_chars)}
indToChar = {i:c for i,c in enumerate(book_chars)}

In [78]:
def toString(encoded_text):
    return ''.join([indToChar[i] for i in encoded_text])

In [79]:
def oneHotEncode(x):
    Y = np.zeros((K, len(x)))
    for i,c in enumerate(x):
        Y[charToInd[c],i] = 1
    return Y

In [80]:
class RNN():
    def __init__(self, K, m=100, seed=123456789):
        np.random.seed(seed)

        self.K = K
        self.m = m
        self.sigma = 0.01

        self.weights = {}
        self.momentum = {}
        
        # Biases
        self.weights["b"] = np.zeros(shape=(self.m,1))
        self.weights["c"] = np.zeros(shape=(K,1))

        # Weights
        self.weights["U"] = np.random.randn(self.m, self.K) * self.sigma
        self.weights["W"] = np.random.randn(self.m, self.m) * self.sigma
        self.weights["V"] = np.random.randn(self.K, self.m) * self.sigma

        # Momentum
        for key, value in self.weights.items():
            self.momentum[key] = np.zeros(value.shape)

        # Set initial hidden state
        self.h0 = np.zeros(shape=(self.m,1))
    
    def cost(self, X, Y):
        P = self.forward(X)

        loss = -np.sum(Y * np.log(P))
        return loss
    
    def forward(self, X, train=False):

        hList = [self.h0.copy()]
        aList = []
        oList = []
        pList = []

        for x in X.T:
            a = self.weights["W"] @ hList[-1] + self.weights["U"] @ x.reshape(-1,1) + self.weights["b"]
            h = np.tanh(a)
            o = self.weights["V"] @ h + self.weights["c"]
            p = np.exp(o) / np.sum(np.exp(o), axis=0)

            hList.append(h)
            aList.append(a)
            oList.append(o)
            pList.append(p)

            H = np.hstack(hList)
            A = np.hstack(aList)
            O = np.hstack(oList)
            P = np.hstack(pList)
        
        if train:
            self.h0 = H[:, -1].reshape(-1,1)
            # P: K x seq_length, H: m x seq_length+1, A: m x seq_length, O: K x seq_length
            return P, H, A, O
        else:
            return P
    
    def backward(self, X, Y):
            
            P, H, A, O = self.forward(X, train=True)
    
            g = P - Y #gradO
            gV = g @ H.T[1:]
            gc = np.sum(g, axis = 1).reshape(-1,1)
    
            gH = g.T[-1] @ self.weights["V"]
            gA = gH * (1 - np.square(np.tanh(A.T[-1])))

            lH = [gH]
            lA = [gA]
    
            # Page 42
            for gt, at in zip(g.T[-2::-1], A.T[-2::-1]):
                gH = gt @ self.weights["V"] + gA @ self.weights["W"]
                gA = gH * (1 - np.square(np.tanh(at)))

                lH.append(gH)
                lA.append(gA)

            gH = np.vstack(lH[::-1]).T
            gA = np.vstack(lA[::-1]).T

            gW = gA @ H.T[:-1]
            gU = gA @ X.T
            gb = np.sum(gA, axis = 1).reshape(-1,1)

            return {"W":gW, "U":gU, "V":gV, "b":gb, "c":gc}

    
    def synth(self, x0, n):

        h = self.h0
        x = x0

        for i in range(n):
            a = self.weights["W"] @ h + self.weights["U"] @ x[:,-1].reshape(-1,1) + self.weights["b"]
            h = np.tanh(a)
            o = self.weights["V"] @ h + self.weights["c"]
            p = np.exp(o) / np.sum(np.exp(o), axis=0)
            idx = np.random.choice(range(self.K),p=np.squeeze(p))
            newX = np.zeros(shape=(self.K,1))
            newX[idx,0] = 1
            x = np.c_[x,newX]
        
        return [np.argmax(c) for c in x.T]
    
    def computeGradsNumerical(self, X, Y, eps):
        grads = {}
        for name, weight in self.weights.items():
            shape = weight.shape
            w_perturb = np.zeros(shape)
            w_gradsNum = np.zeros(shape)
            w_0 = weight.copy()
            
            for i in range(shape[0]):
                for j in range(shape[1]):

                    # add perturbation
                    w_perturb[i, j] = eps
                    
                    # perturb weight vector negatively
                    # and compute cost
                    w_tmp = w_0 - w_perturb
                    self.weights[name] = w_tmp
                    cost1 = self.cost(X, Y)
                
                    # perturb weight vector positively
                    # and compute cost
                    w_tmp = w_0 + w_perturb
                    self.weights[name] = w_tmp
                    cost2 = self.cost(X, Y)
                    lossDiff = (cost2 - cost1) / (2 * eps)
                    
                    # get numerical grad f. W[i, j]
                    w_gradsNum[i, j] = lossDiff
                    w_perturb[i, j] = 0
        
            # save grads
            grads[name] = w_gradsNum
            
            # reset
            self.weights[name] = w_0
            
        return grads
        




In [81]:
m = 100
eta = 0.1
seq_length = 25

# 0.3

In [82]:
x = oneHotEncode(["a"])
model = RNN(K, m)

In [83]:
toString(model.synth(x, 20))

'a:N\nyerF4FY,^•d u,•wl'

# 0.4

In [84]:
def relerr(ga, gn, eps=1e-6):
        """
        Calculates the relative error between two vectors.

        Args:
            ga (numpy.ndarray): Analytical gradient.
            gn (numpy.ndarray): Numerical gradient.
            eps (float, optional): A small value to avoid division by zero. Defaults to 1e-6.

        Returns:
            float: The relative error between ga and gn.
        """
        
        diff = np.linalg.norm(ga - gn)
        norma = np.linalg.norm(ga)
        normn = np.linalg.norm(gn)
        numer = max(eps, norma + normn)
        return diff / numer

In [85]:
Xchars = data[0:seq_length]
Ychars = data[1:seq_length+1]

X = oneHotEncode(Xchars) # K x seq_length
Y = oneHotEncode(Ychars) # K x seq_length

In [86]:
P, H, A, O = model.forward(X, train=True)

In [87]:
P.shape, H.shape, A.shape, O.shape
# P: K x seq_length, H: m x seq_length+1, A: m x seq_length, O: K x seq_length

((80, 25), (100, 26), (100, 25), (80, 25))

In [88]:
angrads = model.backward(X, Y)

In [89]:
numgrads = model.computeGradsNumerical(X, Y, 1e-5)

In [94]:
for key in numgrads.keys():
    print(key)
    print(relerr(angrads[key], numgrads[key]))

b
4.522574447354954e-09
c
4.0230462054063615e-10
U
2.2198521028447137e-08
W
6.180653160583247e-07
V
5.86874726731972e-08
